In [1]:
import sys
import ROOT as R       # Make sure the ROOT system is properly initialized or ROOT will not be found.
R.EnableImplicitMT()   # Note: Comment this if you want to use df.Display().
# Comment the next line this if you don't like the interactive histograms.
%jsroot on  
# Load the MiniDST Library. Not really needed, but can be handy in event by event checking of data and getting constants.
R.gSystem.Load("libMiniDst.dylib")
MiniDst = R.MiniDst()
print("Version of MiniDst Class:",MiniDst._version_())
# Load the Moller library and instantiate the Moller class object.
R.gSystem.Load("libMoller.dylib")
Moller = R.Moller()
print("Version of Moller Class:",Moller.GetVersion())

Version of MiniDst Class: 1.2.2
Version of Moller Class: 1.0


# Mollers
## Introduction
This file is the *example* file for studying Møllers in the HPS data and Monte Carlo files based on the *minidst* format.

The code also makes use of the Moller helper library, which is in the [hps-analysis](https://github.com/JeffersonLab/hps-analysis.git) package on Github. 

## Notes:
* The hps-analysis package, which contains the Minidst format and related code is found here: https://github.com/JeffersonLab/hps-analysis.git


In [2]:
# Get some data. Path will depend on your system. 
# This is the 2016 run 7966 data from the pass 4 Kalman Filter processing.
data_files_pattern = "/data/HPS/data/physrun2016/Moller/pass4kf/hps_007966.*_recon_5.2.1_minidst.root"
Beam_Energy = 2.3
ch_moller = R.TChain("MiniDST")
ch_moller.Add(data_files_pattern)
n_events = ch_moller.GetEntries()
nf = ch_moller.GetListOfFiles().GetEntries()
df = R.RDataFrame(ch_moller)
print(f"Added {nf} files to the chain, total of {n_events/1e6:7.2f}M events")

Added 11 files to the chain, total of    5.14M events


In [3]:
# This is a list of the vertex (v0) particle types from LCIO. 
ParticleNames=[ str(x) for x in MiniDst.ParticleTypeNames]
# This makes a nicely formatted list of all the items in the dataframe. Note that you can just call df.Describe() but that gives a single column.
All_Names = list(df.GetColumnNames())
Store_Names = []
ii=0
remove_names = [] # ["v0_","hodo_","ext_trigger","part_","rf_","svt"]
for n in All_Names:
    s = str(n)
    if not any([s.lower().startswith(x) for x in remove_names]):  # Allow for a filter.
        Store_Names.append(s)                                     # Store_Names can be handy if you later want to write out the filtered data.
        dat_type = df.GetColumnType(s).replace("ROOT::VecOps::RVec<","RVec<")
        print(f"{s:<30s} [{dat_type:<20s}]", end = " ")
        ii += 1
        if ii%2==0:     # Replace the 2 with 3 to get 3 columns (too wide for my screen)
            print("");


ecal_cluster_energy            [RVec<double>        ] ecal_cluster_hits              [RVec<vector<int>>   ] 
ecal_cluster_nhits             [RVec<int>           ] ecal_cluster_seed_energy       [RVec<double>        ] 
ecal_cluster_seed_index        [RVec<int>           ] ecal_cluster_seed_ix           [RVec<int>           ] 
ecal_cluster_seed_iy           [RVec<int>           ] ecal_cluster_time              [RVec<double>        ] 
ecal_cluster_x                 [RVec<double>        ] ecal_cluster_y                 [RVec<double>        ] 
ecal_cluster_z                 [RVec<double>        ] ecal_hit_energy                [RVec<double>        ] 
ecal_hit_index_x               [RVec<int>           ] ecal_hit_index_y               [RVec<int>           ] 
ecal_hit_time                  [RVec<double>        ] ecal_hit_x                     [RVec<double>        ] 
ecal_hit_y                     [RVec<double>        ] ecal_hit_z                     [RVec<double>        ] 
event_number       

In [9]:
# Make a plot of the Vertexes found in this data set. This tests if the data was written correctly with KF tracking.
h_v0_type = df.Histo1D(("h_v0_type","V0 Type",17,-0.5,16.5),"v0_type")
h_ntrk_kf = df.Histo1D(("h_ntrk_kf","Num KF tracks per event",16,-0.5,15.5),"track_n_kf")
h_ntrk_gbl = df.Histo1D(("h_ntrk_gbl","Num GBL tracks per event",16,-0.5,15.5),"track_n_gbl")
h_part_n_ele = df.Define("nele","int out=0; for(int i: part_pdg) if(i==11) out++; return out;").Histo1D(("h_part_n_ele","Num Electrons",16,-0.5,15.5),"nele")
h_part_n_ucm = df.Define("nucm","int out=0; for(int v: v0_type) if(v == 4) out++; return out;").Histo1D(("h_part_n_ucm","Num V0 Unconstrained",16,-0.5,15.5),"nucm")
h_part_n_ele_type4 = df.Define("nele4","int out=0; for(int v: v0_type) if(v == 4) for(int i: part_pdg) if(i==11) out++; return out; ").Histo1D(("h_part_n_ele","Num Electrons for v0 type==4",16,-0.5,15.5),"nele4")
#("h_n_v0","n_v0",11,-0.5,10.5)
cc1 = R.TCanvas("cc1","cc1",1000,500)
cc1.Divide(2,2)
pad1= cc1.cd(1)
pad1.SetBottomMargin(0.2)
axis = h_v0_type.GetXaxis()
axis.SetLabelSize(0.035)
for i in range (0,16):
    axis.SetBinLabel(i+1,f"{i}-{ParticleNames[i]}")
h_v0_type.SetStats(0)
h_v0_type.SetFillColor(R.kBlue-10)
h_v0_type.Draw()
pad2 = cc1.cd(2)
pad2.SetLogy(1)
h_ntrk_gbl.SetLineColor(R.kBlue)
h_ntrk_gbl.SetMinimum(1)
h_ntrk_gbl.Draw("same")
h_ntrk_kf.SetLineColor(R.kGreen)
h_ntrk_kf.Draw("same")
pad3 = cc1.cd(3)
pad3.SetLogy(1)
h_part_n_ele.SetLineColor(R.kGreen)
h_part_n_ele.Draw()
h_part_n_ele_type4.SetLineColor(R.kBlue)
h_part_n_ele_type4.Draw("same")
h_part_n_ucm.SetLineColor(R.kRed)
h_part_n_ucm.Draw("same")
cc1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc1


In the above histograms, the first (top left) shows the assignment of the V0 vertexes that were found by the hps-java code. The column 4-UC Moller Vertex KF shows the number of events with a v0_type == 4 vertex found, i.e. 2 electrons. Note that quite a few events have e+ e- pairs. The second histogram confirms that KF tracking was used (lots of tracks for KF) and that GBL tracking was off, either when producing the SLCIO file (likely), or when converting that file to the minidst format (possible, but not the case here.)

The last histogram shows the number of electrons (green) and the number of type==4 vertexes found (red). Since there was never more than one vertex found, this data was processed with ***the old code*** for finding Moller vertexes, which required one and only one electron in the top and one and only one electron in the bottom of the detector. If two or more were found the event was skipped. The blue curve on that plot confirms this: an event with a v0_type==4 has only two electrons.

## Moller Data Selection
We create some distrubutions from the data in dataframes. First is data with two electrons that have a track time that coincides within 2 ns, which makes it likely these are Moller pairs. For background events we take two electrons that have a track time more than 10 ns apart. We also create dataframes that have an unconstrained moller vertex, a beamspot constrained moller vertex and target constrained moller vertex identified.


In [12]:
Beam_Energy = 2.3  ## Store the beam energy for these runs. This is convenient to have in the notebook.
dfee = Moller.Select_El_Pairs(R.RDF.AsRNode(df), 0., 2.,"electron_pairs") \
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")
# Select delta time < 2ns.
dfbck = Moller.Select_El_Pairs(R.RDF.AsRNode(df),10., 100., "electron_pairs") \
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")  # Select delta time > 4ns and < 20ns.

dfucm = Moller.Select_v0(R.RDF.AsRNode(df), 4 , 0., 10., "electron_pairs") \
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")
dfbcm = Moller.Select_v0(R.RDF.AsRNode(df), 5 , 0., 10., "electron_pairs") \
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")
dftcm = Moller.Select_v0(R.RDF.AsRNode(df), 6 , 0., 10., "electron_pairs") \
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")

# Count numbers of events
n_events = df.Count()       # Any event
n_events_ee = dfee.Count()  # Events with at least one ee pair in time.
n_events_bck = dfbck.Count() # Events with at least one ee pair out of time
n_events_ucm = dfucm.Count() # Events with an UC Moller vertex
n_events_bcm = dfbcm.Count() # Events with an BC Moller vertex
n_events_tcm = dfbcm.Count() # Events with an TC Moller vertex

print("Events Summary.")
print(f"Total number of events: {n_events.GetValue()}")
print(f"Events with ee:         {n_events_ee.GetValue()}")
print(f"Events with bck:        {n_events_bck.GetValue()}")
print(f"Events with UC Moller:  {n_events_ucm.GetValue()}")
print(f"Events with BC Moller:  {n_events_bcm.GetValue()}")
print(f"Events with TC Moller:  {n_events_tcm.GetValue()}")


Events Summary.
Total number of events: 5139209
Events with ee:         440654
Events with bck:        2242298
Events with UC Moller:  388146
Events with BC Moller:  388146
Events with TC Moller:  388146


## Moller theory
Moller scattering is just electron-electron scattering on (at these energies) nearly free electrons. The kinematics can thus be completely computed. 
You can find a complete derivation of the cross-section here: [Moller Scattering](https://www-user.rhrk.uni-kl.de/~apelster/Vorlesungen/WS2021/v11.pdf)

The invariant mass of the moller pair can be computed from:

$$ s = (p_1 + p_2)^2 = 2 M_e^2 + 2 E_0 M_e = M_{inv}^2 $$

The energy-angle relation of the Moller electrons is given by:
$$E(\theta) = \frac{E_0}{1+\frac{2E_0}{m_e}\sin^2 \left( \theta/2\right)}$$

The angle of the one Moller electrons knowing the other angle is given by:

$$\theta_2 \left(\theta_1 \right) = 2\sin^{-1}\left( \frac{m_e}{2 E_0 \sin(\theta_1 /2)} \right)$$

## Moller Selection - Refinement cuts.

For each of the dataframes we can now add a set of cuts. The cuts in the "R" series are likely okay to make. The cuts in the 
"X" series cut on the angles in ways that may affect the width of the Moller peak. We start with the selection of two electrons 

The cuts are as follows:

* R1 - Cut on the electron momentum $|p_e| < 2 $ GeV
* R2 - Cut on the $\phi$ angle. If angle not specified, require the electrons to be in opposite volumes of HPS. With angle specified: $ |\phi_{e1} - \phi_{e2}| < \phi_{max}$.
* R3 - Cut on the calculated beam angle. $ (\vec{p}_{e1} + \vec{p}_{e2})_{\theta} < \theta_{max}$.
* R4 - Cut on the 3-momentum sum of the two electrons. $ 0.75 E_{b} < | \vec{p}_{e1} + \vec{p}_{e2} | < 1.15 E_{b}$.
* X1 - Brigid's cut on the sum of the theta angles, $ \theta_{min} < \theta_{e1} + \theta_{e2} < \theta_{max}$.
* X2 - Omar's fiducial cut on the tracks at the ECal, see Analysis note.

After each cut a standard set of variables is added to the dataframe in the Add_Four_Vectors method. This adds the 4-vectors for each electron rotated to account for the beam direction, the invariant mass of the 2 electrons etc. In some cuts above these variables are used, so we need to be careful to add them after a cut is made, and we need to make sure the ordering is correct.

Each of these refinement cuts is implemented for dataframes in the Moller class that we loaded at the top of the notebook. The class is designed to work with the pure e-e events and nearly transparently with the v0 vertexed events.


In [16]:
d = [dfee, dfbck, dfucm, dfbcm, dftcm]

for i in range(len(d)):
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "electron_pairs","part_","p4")
    # This cut simply cuts out the FEE electrons, and does not (well, really widely) cut the momentum sum.
    d[i] = Moller.Refine_El_Pairs_1(R.RDF.AsRNode(d[i]), 0.8*Beam_Energy, Beam_Energy*0., Beam_Energy*2., "electron_pairs", "electron_pairs_idx", "el_r1")
    # This cut chooses opposite halves of the detector for the two electrons. A more stringent cut can be made with the commented out line, cutting on
    # 30 degree co-planarity.
    # d[i] = Moller.Refine_El_Pairs_2(R.RDF.AsRNode(d[i]), 30./180.*R.Math.Pi() , "electron_pairs", "el_r1", "el_r2")
    d[i] = Moller.Refine_El_Pairs_2(R.RDF.AsRNode(d[i]),  "electron_pairs", "el_r1", "el_r2")
    # This cuts on the moller angle being coincident with the beam angle.  
    d[i] = Moller.Refine_El_Pairs_3(R.RDF.AsRNode(d[i]), 10./180.*R.Math.Pi(), "el_r2", "p4", "el_r3")
    # This cut cuts on the momentum sum spectrum.
    d[i] = Moller.Refine_El_Pairs_1(R.RDF.AsRNode(d[i]), 0.8*Beam_Energy, Beam_Energy*0.85, Beam_Energy*1.1,  "electron_pairs", "el_r3", "el_r4")

    # These cuts should not be done.
    d[i] = Moller.Refine_El_Pairs_X(R.RDF.AsRNode(d[i]), 0.040, 0.048, "p4tht1", "p4tht2" ,"el_r4", "el_rx1")
    d[i] = Moller.Refine_El_Pairs_X2(R.RDF.AsRNode(d[i]),"electron_pairs", "el_r4", "el_rx2")

dfee, dfbck, dfucm, dfbcm, dftcm = d # Reverse assign.

## Data Check
We take a look at the distributions that we get for some of the Moller events that we found.


In [24]:
# Define the histograms
h_n_part = dfee.Define("n_part","return part_pdg.size()").Histo1D(("h_n_part","Number of particles",11,-0.5,10.5),"n_part")
h_n_neg_track = dfee.Define("n_neg_track","int count=0;for(int i=0;i<track_omega.size();++i){if(track_omega[i]>0 && track_type[i]==1)count++;} return count;").Histo1D(("h_n_neg_track","Number of - tracks",11,-0.5,10.5),"n_neg_track")
h_n_electron = dfee.Define("n_electron","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Histo1D(("h_n_electron","Number of e-",11,-0.5,10.5),"n_electron")
h_n_moller = dfee.Define("n_moller","return electron_pairs.size()").Histo1D(("h_n_moller","Number of Moller pairs",11,-0.5,10.5),"n_moller")

h_track_time_diff = dfee.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("h_track_time_diff","track_time_diff",500,-20.,20.),"track_time_diff")
hx_track_time_diff = dfbck.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("hx_track_time_diff","track_time_diff",500,-20.,20.),"track_time_diff")
hm_track_time_diff = dfucm.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("hm_track_time_diff","track_time_diff",500,-20.,20.),"track_time_diff")

h_track_chi2 = dfee.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("h_track_chi2","track_chi2",500,0.,10.),"t_chi2")
hx_track_chi2 = dfbck.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("xh_track_chi2","track_chi2",500,0.,10.),"t_chi2")

h_track_nhit = dfee.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("h_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")
hx_track_nhit = dfbck.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("hx_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")

h_mom_sum = dfee.Histo1D(("h_mom_sum","mom sum",500,0.,6.0),"p4psum")
h_mom_sum_r1 = dfee.Define("mom_sum","vector<double> out; for(int ind: el_r1){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r1","mom sum r1",500,0.,6.0),"mom_sum")
h_mom_sum_r2 = dfee.Define("mom_sum","vector<double> out; for(int ind: el_r2){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r2","mom sum r1",500,0.,6.0),"mom_sum")
h_mom_sum_r3 = dfee.Define("mom_sum","vector<double> out; for(int ind: el_r3){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r3","mom sum r1",500,0.,6.0),"mom_sum")
h_mom_sum_r4 = dfee.Define("mom_sum","vector<double> out; for(int ind: el_r4){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r4","mom sum r1",500,0.,6.0),"mom_sum")

hx_mom_sum = dfbck.Histo1D(("h_mom_sum","mom sum",500,0.,6.0),"p4psum")
hx_mom_sum_r1 = dfbck.Define("mom_sum","vector<double> out; for(int ind: el_r1){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r1","mom sum r1",500,0.,6.0),"mom_sum")
hx_mom_sum_r2 = dfbck.Define("mom_sum","vector<double> out; for(int ind: el_r2){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r2","mom sum r1",500,0.,6.0),"mom_sum")
hx_mom_sum_r3 = dfbck.Define("mom_sum","vector<double> out; for(int ind: el_r3){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r3","mom sum r1",500,0.,6.0),"mom_sum")
hx_mom_sum_r4 = dfbck.Define("mom_sum","vector<double> out; for(int ind: el_r4){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r4","mom sum r1",500,0.,6.0),"mom_sum")


hm_mom_sum = dfucm.Histo1D(("h_mom_sum","mom sum",500,0.,6.0),"p4psum")
hm_mom_sum_r1 = dfucm.Define("mom_sum","vector<double> out; for(int ind: el_r1){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r1","mom sum r1",500,0.,6.0),"mom_sum")
hm_mom_sum_r2 = dfucm.Define("mom_sum","vector<double> out; for(int ind: el_r2){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r2","mom sum r1",500,0.,6.0),"mom_sum")
hm_mom_sum_r3 = dfucm.Define("mom_sum","vector<double> out; for(int ind: el_r3){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r3","mom sum r1",500,0.,6.0),"mom_sum")
hm_mom_sum_r4 = dfucm.Define("mom_sum","vector<double> out; for(int ind: el_r4){out.push_back(p4psum[ind]);} return out;").Histo1D(("h_mom_sum_r4","mom sum r1",500,0.,6.0),"mom_sum")


h_beam_theta_x = dfee.Histo1D(("h_beam_theta_x","Beam Angle #theta_{x}",1000,-0.1,0.1),"p4beam_theta_x")
h_beam_theta_x_r2 = dfee.Define("beam_theta_x","vector<double> out; for(int ind: el_r2){out.push_back(p4beam_theta_x[ind]);} return out;").Histo1D(("h_beam_theta_x_r2","Beam Angle #theta_{x}",1000,-0.1,0.1),"beam_theta_x")
h_beam_theta_x_r3 = dfee.Define("beam_theta_x","vector<double> out; for(int ind: el_r3){out.push_back(p4beam_theta_x[ind]);} return out;").Histo1D(("h_beam_theta_x_r3","Beam Angle #theta_{x}",1000,-0.1,0.1),"beam_theta_x")
h_beam_theta_x_r4 = dfee.Define("beam_theta_x","vector<double> out; for(int ind: el_r4){out.push_back(p4beam_theta_x[ind]);} return out;").Histo1D(("h_beam_theta_x_r4","Beam Angle #theta_{x}",1000,-0.1,0.1),"beam_theta_x")

hx_beam_theta_x = dfbck.Histo1D(("hx_beam_theta_x","Beam Angle #theta_{x}",1000,-0.1,0.1),"p4beam_theta_x")

h_ee_thtE1 = dfee.Histo2D(("h_ee_thtE1","EE theta-E 1;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"p4tht1","p4E1")
h_ee_thtE2 = dfee.Histo2D(("h_ee_thtE2","EE theta-E 2;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"p4tht2","p4E2")
h_ee_thth = dfee.Histo2D(("h_ee_thth","EE theta-theta;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"p4tht1","p4tht2")
#h_ee_thtE1_r1 = dfee.Histo2D(("h_ee_thtE1","EE theta-E 1 R1;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"r1_p4tht1","r1_p4E1")
#h_ee_thtE2_r1 = dfee.Histo2D(("h_ee_thtE2","EE theta-E 2 R1;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"r1_p4tht2","r1_p4E2")
#h_ee_thth_r1 = dfee.Histo2D(("h_ee_thth","EE theta-theta R1;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"r1_p4tht1","r1_p4tht2")
h_ee_thtE1_r2 = dfee.Define("tht1","vector<double> out; for(int ind: el_r2){out.push_back(p4tht1[ind]);} return out;").Define("E1","vector<double> out; for(int ind: el_r2){out.push_back(p4E1[ind]);} return out;").Histo2D(("h_ee_thtE1_r2","EE theta-E 1 R2;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"tht1","E1")
h_ee_thtE2_r2 = dfee.Define("tht2","vector<double> out; for(int ind: el_r2){out.push_back(p4tht2[ind]);} return out;").Define("E2","vector<double> out; for(int ind: el_r2){out.push_back(p4E2[ind]);} return out;").Histo2D(("h_ee_thtE2_r2","EE theta-E 1 R2;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"tht2","E2")
h_ee_thth_r2 = dfee.Define("tht1","vector<double> out; for(int ind: el_r2){out.push_back(p4tht1[ind]);} return out;").Define("tht2","vector<double> out; for(int ind: el_r2){out.push_back(p4tht2[ind]);} return out;").Histo2D(("h_ee_thth_r2","EE theta-theta R2;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"tht1","tht2")
h_ee_thtE1_r3 = dfee.Define("tht1","vector<double> out; for(int ind: el_r3){out.push_back(p4tht1[ind]);} return out;").Define("E1","vector<double> out; for(int ind: el_r3){out.push_back(p4E1[ind]);} return out;").Histo2D(("h_ee_thtE1_r3","EE theta-E 1 R2+R3;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"tht1","E1")
h_ee_thtE2_r3 = dfee.Define("tht2","vector<double> out; for(int ind: el_r3){out.push_back(p4tht2[ind]);} return out;").Define("E2","vector<double> out; for(int ind: el_r3){out.push_back(p4E2[ind]);} return out;").Histo2D(("h_ee_thtE2_r3","EE theta-E 1 R2+R3;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"tht2","E2")
h_ee_thth_r3 = dfee.Define("tht1","vector<double> out; for(int ind: el_r3){out.push_back(p4tht1[ind]);} return out;").Define("tht2","vector<double> out; for(int ind: el_r3){out.push_back(p4tht2[ind]);} return out;").Histo2D(("h_ee_thth_r3","EE theta-theta R2+R3;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"tht1","tht2")


In [26]:
# Turn on/off the dynamic histograms.
%jsroot on
cc0 = R.TCanvas("cc0","cc0",1200,800)
cc0.Divide(3,2)
cc0.cd(1)
h_n_moller.SetTitle("Particle counts in Moller events")
h_n_moller.SetStats(0)
h_n_moller.SetLineWidth(2)
h_n_moller.SetLineColor(R.kCyan)
h_n_moller.Draw("same")
h_n_electron.SetLineWidth(3)
h_n_electron.SetLineColor(R.kBlue)
h_n_electron.Draw("same")
h_n_neg_track.SetLineColor(R.kRed)
h_n_neg_track.Draw("same")
h_n_part.SetLineWidth(1)
h_n_part.SetLineColor(R.kGreen+2)
h_n_part.Draw("same")
legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_n_part.GetPtr(),"any particle","l")
legend.AddEntry(h_n_neg_track.GetPtr(),"neg. tracks","l")
legend.AddEntry(h_n_electron.GetPtr(),"electron","l")
legend.AddEntry(h_n_moller.GetPtr(),"Moller","l")
legend.Draw()
cc0.cd(2)
h_track_time_diff.SetLineWidth(1)
h_track_time_diff.SetLineColor(R.kGreen+2)
h_track_time_diff.Draw()
hx_track_time_diff.SetLineColor(R.kRed)
hx_track_time_diff.Draw("same")
# hm_track_time_diff.SetLineColor(R.kAzure)
# hm_track_time_diff.Draw("same")
pad6 = cc0.cd(3)
pad6.SetLogy()
hx_beam_theta_x.SetStats(0)
hx_beam_theta_x.SetLineColor(R.kRed)
hx_beam_theta_x.Draw("same")
#hx_beam_theta_x.Fit("gaus")

h_beam_theta_x.SetStats(0)
h_beam_theta_x.SetLineWidth(1)
h_beam_theta_x.SetLineColor(R.kGreen+2)
h_beam_theta_x.Draw("same")
#h_beam_theta_x.Fit("gaus")

h_beam_theta_x_r2.SetStats(0)
h_beam_theta_x_r2.SetLineWidth(1)
h_beam_theta_x_r2.SetLineColor(R.kBlue)
h_beam_theta_x_r2.Draw("same")
#h_beam_theta_x_r2.Fit("gaus")

h_beam_theta_x_r4.SetStats(0)
h_beam_theta_x_r4.SetLineWidth(1)
h_beam_theta_x_r4.SetLineColor(R.kOrange)
h_beam_theta_x_r4.Draw("same")
# #h_beam_theta_x_r3.Fit("gaus")

leg2 = R.TLegend(0.7,0.7,0.9,0.9)
leg2.AddEntry(hx_beam_theta_x.GetPtr(),"Not Moller","l")
leg2.AddEntry(h_beam_theta_x.GetPtr(),"Moller","l")
leg2.AddEntry(h_beam_theta_x_r2.GetPtr(),"Moller r2","l")
leg2.AddEntry(h_beam_theta_x_r4.GetPtr(),"Moller r2+r3+r4","l")
leg2.Draw()

pad2 = cc0.cd(4)
pad2.SetLogz()
h_ee_thth_r3.SetStats(0)
h_ee_thth_r3.Draw("colz")
l1 = R.TLine(0.,0.04,0.04,0.)
# l1.SetLineColor(R.kRed)
# l1.SetLineWidth(2)
# l1.Draw()
# l2 = R.TLine(0.,0.048,0.048,0.)
# l2.SetLineColor(R.kRed)
# l2.SetLineWidth(2)
# l2.Draw()
# tht2(tht1) = 2*asin(m_e/(2*E0)/sin(tht1/2))
f1 = R.TF1("f1","2*asin(0.000511/(2*2.3)/sin(x/2))",0.0044,0.1)
f1.SetLineColor(R.kGreen)
f1.Draw("same")

pad3 = cc0.cd(5)
pad3.SetLogz()
h_ee_thtE1_r3.SetStats(0)
h_ee_thtE1_r3.Draw("colz")
f2 = R.TF1("f2","2.3/(1+ (2.3/0.000511)*(1-cos(x)))",0.005,0.1)
f2.SetLineColor(R.kRed)
f2.Draw("same")

pad4 = cc0.cd(6)
#f1 = R.TF1("f1","2*asin(0.11108696/sin(x/2))",0.0001,0.1)
pad4.SetLogz()
h_ee_thtE2_r3.SetStats(0)
h_ee_thtE2_r3.Draw("colz")
f3 = R.TF1("f3","2.3/(1+ (2.3/0.000511)*(1-cos(x)))",0.005,0.1)
f3.SetLineColor(R.kRed)
f3.Draw("same")


cc0.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0
